In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np

PROJECT_ROOT = "/content/drive/MyDrive/ds-mini-project-2025"  # change if yours differs

RAW_DIR = f"{PROJECT_ROOT}/data_raw"
CLEAN_DIR = f"{PROJECT_ROOT}/data_clean"
DOCS_DIR = f"{PROJECT_ROOT}/docs"
FIG_DIR = f"{PROJECT_ROOT}/figures"

for p in [RAW_DIR, CLEAN_DIR, DOCS_DIR, FIG_DIR]:
    os.makedirs(p, exist_ok=True)

print("RAW_DIR:", RAW_DIR)
print("CLEAN_DIR:", CLEAN_DIR)
print("DOCS_DIR:", DOCS_DIR)
print("FIG_DIR:", FIG_DIR)

RAW_DIR: /content/drive/MyDrive/ds-mini-project-2025/data_raw
CLEAN_DIR: /content/drive/MyDrive/ds-mini-project-2025/data_clean
DOCS_DIR: /content/drive/MyDrive/ds-mini-project-2025/docs
FIG_DIR: /content/drive/MyDrive/ds-mini-project-2025/figures


In [3]:
week6_candidates = [
    f"{CLEAN_DIR}/event_week6_profiled.csv",
    f"{CLEAN_DIR}/event_clean.csv",
    f"{RAW_DIR}/event_practice.csv",
    f"{RAW_DIR}/event.csv",
]

input_path = None
for p in week6_candidates:
    if os.path.exists(p):
        input_path = p
        break

assert input_path is not None, "No input CSV found. Put event_practice.csv in data_raw/ first."

df = pd.read_csv(input_path)
print("Loaded:", input_path)
print("Shape:", df.shape)
df.head()

Loaded: /content/drive/MyDrive/ds-mini-project-2025/data_clean/event_clean.csv
Shape: (25, 12)


,ticket_id,event_id,event_topic,buyer_id,purchase_date,qty,price_usd,channel,channel_clean,price_usd_clean,purchase_date_parsed,event_topic_clean
0,T01001,E002,Ai,B0013,2/5/2025,1,32.07,web,web,32.07,2025-02-05,AI
1,T01002,E003,Cloud,B0017,NaN,2,14.31,app,app,14.31,NaN,Cloud
2,T01003,E004,IoT,B0215,1/2/2025,1,32.93,app,app,32.93,2025-01-02,IoT
3,T01004,E005,Data Engineering,B0173,4/2/2025,3,29.15,on_site,onsite,29.15,2025-04-02,Data Engineering
4,T01004,E006,IoT,B0064,5/22/2025,2,38.18,onsite,onsite,38.18,2025-05-22,IoT


---
## Part 1 — Quick Inspect (Week 5 habit)


In [4]:
def missingness_table(df):
    out = pd.DataFrame({
        "missing_count": df.isna().sum(),
        "missing_pct": (df.isna().mean() * 100).round(2),
        "dtype": df.dtypes.astype(str)
    }).sort_values(["missing_pct", "missing_count"], ascending=False)
    return out

miss = missingness_table(df)
miss


,missing_count,missing_pct,dtype
purchase_date,3,12.0,object
purchase_date_parsed,3,12.0,object
ticket_id,0,0.0,object
event_id,0,0.0,object
event_topic,0,0.0,object
buyer_id,0,0.0,object
qty,0,0.0,int64
price_usd,0,0.0,object
channel,0,0.0,object
channel_clean,0,0.0,object


In [5]:
FIX_LOG_COLUMNS = [
    "column",
    "problem_found",
    "rule_applied",
    "kept_original_column",
    "new_column_name",
    "missing_strategy",
    "assumptions_notes"
]

fix_log = pd.DataFrame(columns=FIX_LOG_COLUMNS)

def add_fix(column, problem_found, rule_applied, kept_original_column, new_column_name,
            missing_strategy, assumptions_notes):
    global fix_log
    fix_log = pd.concat([fix_log, pd.DataFrame([{
        "column": column,
        "problem_found": problem_found,
        "rule_applied": rule_applied,
        "kept_original_column": kept_original_column,
        "new_column_name": new_column_name,
        "missing_strategy": missing_strategy,
        "assumptions_notes": assumptions_notes
    }])], ignore_index=True)

fix_log


,column,problem_found,rule_applied,kept_original_column,new_column_name,missing_strategy,assumptions_notes


In [6]:
df_ext = df.copy()

rng = np.random.default_rng(7)
n = len(df_ext)
if n >= 30:
    idx = rng.choice(df_ext.index, size=min(25, n//3), replace=False)
else:
    idx = df_ext.index

# Inject missingness
if "purchase_date" in df_ext.columns:
    df_ext.loc[idx[:8], "purchase_date"] = None

if "buyer_id" in df_ext.columns:
    df_ext.loc[idx[8:12], "buyer_id"] = None

# Inject unit/type mess into price_usd
if "price_usd" in df_ext.columns:
    price_as_str = df_ext["price_usd"].astype(str)
    # some $ and USD
    df_ext.loc[idx[12:16], "price_usd"] = "$" + price_as_str.loc[idx[12:16]]
    df_ext.loc[idx[16:20], "price_usd"] = "USD " + price_as_str.loc[idx[16:20]]
    # some KHR values (teaching-only)
    df_ext.loc[idx[20:23], "price_usd"] = "40000 KHR"

# Inject qty as messy strings (forces object dtype when reloaded)
if "qty" in df_ext.columns:
    df_ext.loc[idx[:5], "qty"] = df_ext.loc[idx[:5], "qty"].astype(str) + " tickets"

extended_path = f"{RAW_DIR}/event_week7_extended.csv"
df_ext.to_csv(extended_path, index=False)
print("Saved extended dataset:", extended_path)
df_ext.head()


Saved extended dataset: /content/drive/MyDrive/ds-mini-project-2025/data_raw/event_week7_extended.csv


/tmp/ipython-input-1945008676.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['1 tickets' '2 tickets' '1 tickets' '3 tickets' '2 tickets']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_ext.loc[idx[:5], "qty"] = df_ext.loc[idx[:5], "qty"].astype(str) + " tickets"


,ticket_id,event_id,event_topic,buyer_id,purchase_date,qty,price_usd,channel,channel_clean,price_usd_clean,purchase_date_parsed,event_topic_clean
0,T01001,E002,Ai,B0013,None,1 tickets,32.07,web,web,32.07,2025-02-05,AI
1,T01002,E003,Cloud,B0017,None,2 tickets,14.31,app,app,14.31,NaN,Cloud
2,T01003,E004,IoT,B0215,None,1 tickets,32.93,app,app,32.93,2025-01-02,IoT
3,T01004,E005,Data Engineering,B0173,None,3 tickets,29.15,on_site,onsite,29.15,2025-04-02,Data Engineering
4,T01004,E006,IoT,B0064,None,2 tickets,38.18,onsite,onsite,38.18,2025-05-22,IoT


In [7]:
df = pd.read_csv(f"{RAW_DIR}/event_week7_extended.csv")
print(df.dtypes)
missingness_table(df).head(10)


ticket_id                object
event_id                 object
event_topic              object
buyer_id                 object
purchase_date            object
qty                      object
price_usd                object
channel                  object
channel_clean            object
price_usd_clean         float64
purchase_date_parsed     object
event_topic_clean        object
dtype: object


,missing_count,missing_pct,dtype
purchase_date,9,36.0,object
buyer_id,4,16.0,object
purchase_date_parsed,3,12.0,object
ticket_id,0,0.0,object
event_id,0,0.0,object
event_topic,0,0.0,object
qty,0,0.0,object
price_usd,0,0.0,object
channel,0,0.0,object
channel_clean,0,0.0,object


In [8]:
import re

def normalize_channel(x):
    s = str(x).strip().lower()
    s = s.replace("on_site", "onsite")
    s = s.replace("walk-in", "walkin")
    mapping = {
        "web": "web",
        "app": "app",
        "onsite": "onsite",
        "walkin": "walkin"
    }
    return mapping.get(s, s)  # keep unknowns visible

def normalize_topic(x):
    s = str(x).strip().lower()
    s = re.sub(r"[^a-z0-9\s]+", "", s)  # remove punctuation like A.I.
    s = s.replace("a i", "ai").replace("a  i", "ai")
    mapping = {
        "ai": "ai",
        "cloud": "cloud",
        "iot": "iot",
        "data engineering": "data engineering"
    }
    return mapping.get(s, s)

def parse_qty(x):
    if pd.isna(x):
        return pd.NA
    s = str(x).strip().lower()
    m = re.search(r"\d+", s)
    return int(m.group()) if m else pd.NA

def price_to_usd(x, khr_per_usd=4000):
    """
    Teaching-only:
    - '$12.5', 'USD 12.5' -> 12.5
    - '40000 KHR' -> 40000/khr_per_usd
    """
    if pd.isna(x):
        return np.nan
    s = str(x).strip().upper()
    s = s.replace("$", "").replace("USD", "").strip()
    if "KHR" in s:
        num = re.sub(r"[^0-9.]", "", s.replace("KHR", "").strip())
        try:
            return float(num) / float(khr_per_usd)
        except:
            return np.nan
    # normal numeric
    s = s.replace(",", "")
    try:
        return float(s)
    except:
        return np.nan


In [9]:
df2 = df.copy()

# IDs as string (safe)
for c in ["ticket_id", "event_id", "buyer_id"]:
    if c in df2.columns:
        df2[c] = df2[c].astype("string")

# Standardized categorical columns
if "channel" in df2.columns:
    df2["channel_clean"] = df2["channel"].map(normalize_channel).astype("string")

if "event_topic" in df2.columns:
    df2["event_topic_clean"] = df2["event_topic"].map(normalize_topic).astype("string")

# Standardized qty + price
if "qty" in df2.columns:
    df2["qty_clean"] = df2["qty"].map(parse_qty).astype("Int64")

if "price_usd" in df2.columns:
    df2["price_usd_clean"] = df2["price_usd"].map(price_to_usd)

# Standardized date
if "purchase_date" in df2.columns:
    df2["purchase_date_parsed"] = pd.to_datetime(
        df2["purchase_date"].astype("string"),
        errors="coerce",
        dayfirst=True
    )

df2.head()


,ticket_id,event_id,event_topic,buyer_id,purchase_date,qty,price_usd,channel,channel_clean,price_usd_clean,purchase_date_parsed,event_topic_clean,qty_clean
0,T01001,E002,Ai,B0013,NaN,1 tickets,32.07,web,web,32.07,NaT,ai,1
1,T01002,E003,Cloud,B0017,NaN,2 tickets,14.31,app,app,14.31,NaT,cloud,2
2,T01003,E004,IoT,B0215,NaN,1 tickets,32.93,app,app,32.93,NaT,iot,1
3,T01004,E005,Data Engineering,B0173,NaN,3 tickets,29.15,on_site,onsite,29.15,NaT,data engineering,3
4,T01004,E006,IoT,B0064,NaN,2 tickets,38.18,onsite,onsite,38.18,NaT,iot,2


In [10]:
add_fix(
    column="purchase_date",
    problem_found="Mixed formats + missing values (text dates cannot be used for time analysis).",
    rule_applied="Create purchase_date_parsed using pd.to_datetime(errors='coerce', dayfirst=True).",
    kept_original_column="Yes",
    new_column_name="purchase_date_parsed",
    missing_strategy="Keep missing as NaT + add flag later",
    assumptions_notes="If parsing fails -> NaT (not guessed)."
)

add_fix(
    column="price_usd",
    problem_found="Some values may include symbols/strings (e.g., '$', 'USD', 'KHR') causing object dtype.",
    rule_applied="Create price_usd_clean as float USD using parsing rules.",
    kept_original_column="Yes",
    new_column_name="price_usd_clean",
    missing_strategy="If unparseable -> NaN + flag later",
    assumptions_notes="Teaching-only: 'KHR' converted using fixed khr_per_usd constant."
)

add_fix(
    column="qty",
    problem_found="May contain messy text like '3 tickets' or missing values.",
    rule_applied="Extract digits -> qty_clean (Int64).",
    kept_original_column="Yes",
    new_column_name="qty_clean",
    missing_strategy="If cannot parse -> NA",
    assumptions_notes="Only digits are extracted."
)

add_fix(
    column="channel",
    problem_found="Variants like on_site/onsite/walk-in/walkin.",
    rule_applied="Map variants to {web, app, onsite, walkin} into channel_clean.",
    kept_original_column="Yes",
    new_column_name="channel_clean",
    missing_strategy="If missing -> keep missing for now",
    assumptions_notes="Unknown categories are kept as-is so we can see them."
)

fix_log


,column,problem_found,rule_applied,kept_original_column,new_column_name,missing_strategy,assumptions_notes
0,purchase_date,Mixed formats + missing values (text dates can...,Create purchase_date_parsed using pd.to_dateti...,Yes,purchase_date_parsed,Keep missing as NaT + add flag later,If parsing fails -> NaT (not guessed).
1,price_usd,"Some values may include symbols/strings (e.g.,...",Create price_usd_clean as float USD using pars...,Yes,price_usd_clean,If unparseable -> NaN + flag later,Teaching-only: 'KHR' converted using fixed khr...
2,qty,May contain messy text like '3 tickets' or mis...,Extract digits -> qty_clean (Int64).,Yes,qty_clean,If cannot parse -> NA,Only digits are extracted.
3,channel,Variants like on_site/onsite/walk-in/walkin.,"Map variants to {web, app, onsite, walkin} int...",Yes,channel_clean,If missing -> keep missing for now,Unknown categories are kept as-is so we can se...


In [11]:
df3 = df2.copy()

def add_missing_flag(df, col):
    flag_col = f"{col}_missing"
    df[flag_col] = df[col].isna()
    return flag_col

# Flags
flag_cols = []
for col in ["buyer_id", "purchase_date_parsed", "qty_clean", "price_usd_clean", "channel_clean"]:
    if col in df3.columns:
        flag_cols.append(add_missing_flag(df3, col))

flag_cols


['buyer_id_missing',
 'purchase_date_parsed_missing',
 'qty_clean_missing',
 'price_usd_clean_missing',
 'channel_clean_missing']

In [12]:
# 1) price_usd_clean -> median imputation (only if needed)
if "price_usd_clean" in df3.columns:
    med = df3["price_usd_clean"].median(skipna=True)
    df3["price_usd_filled"] = df3["price_usd_clean"].fillna(med)
    add_fix(
        column="price_usd_clean",
        problem_found="Some missing/unparseable prices.",
        rule_applied="Create price_usd_filled using median(price_usd_clean).",
        kept_original_column="Yes",
        new_column_name="price_usd_filled",
        missing_strategy="Median + keep missing flag",
        assumptions_notes=f"Median used: {med}"
    )

# 2) channel_clean -> mode imputation (optional pattern)
if "channel_clean" in df3.columns:
    mode_val = df3["channel_clean"].mode(dropna=True)
    mode_val = mode_val.iloc[0] if len(mode_val) else "unknown"
    df3["channel_filled"] = df3["channel_clean"].fillna(mode_val)
    add_fix(
        column="channel_clean",
        problem_found="Some missing channels.",
        rule_applied="Create channel_filled using mode(channel_clean).",
        kept_original_column="Yes",
        new_column_name="channel_filled",
        missing_strategy="Mode + keep missing flag",
        assumptions_notes=f"Mode used: {mode_val}"
    )

df3.head()


,ticket_id,event_id,event_topic,buyer_id,purchase_date,qty,price_usd,channel,channel_clean,price_usd_clean,purchase_date_parsed,event_topic_clean,qty_clean,buyer_id_missing,purchase_date_parsed_missing,qty_clean_missing,price_usd_clean_missing,channel_clean_missing,price_usd_filled,channel_filled
0,T01001,E002,Ai,B0013,NaN,1 tickets,32.07,web,web,32.07,NaT,ai,1,False,True,False,False,False,32.07,web
1,T01002,E003,Cloud,B0017,NaN,2 tickets,14.31,app,app,14.31,NaT,cloud,2,False,True,False,False,False,14.31,app
2,T01003,E004,IoT,B0215,NaN,1 tickets,32.93,app,app,32.93,NaT,iot,1,False,True,False,False,False,32.93,app
3,T01004,E005,Data Engineering,B0173,NaN,3 tickets,29.15,on_site,onsite,29.15,NaT,data engineering,3,False,True,False,False,False,29.15,onsite
4,T01004,E006,IoT,B0064,NaN,2 tickets,38.18,onsite,onsite,38.18,NaT,iot,2,False,True,False,False,False,38.18,onsite


In [13]:
before = missingness_table(df)
after = missingness_table(df3)

print("=== BEFORE (top 8) ===")
display(before.head(8))

print("=== AFTER (top 12) ===")
display(after.head(8))


=== BEFORE (top 8) ===


,missing_count,missing_pct,dtype
purchase_date,9,36.0,object
buyer_id,4,16.0,object
purchase_date_parsed,3,12.0,object
ticket_id,0,0.0,object
event_id,0,0.0,object
event_topic,0,0.0,object
qty,0,0.0,object
price_usd,0,0.0,object


=== AFTER (top 12) ===


,missing_count,missing_pct,dtype
purchase_date_parsed,21,84.0,datetime64[ns]
purchase_date,9,36.0,object
buyer_id,4,16.0,string
ticket_id,0,0.0,string
event_id,0,0.0,string
event_topic,0,0.0,object
qty,0,0.0,object
price_usd,0,0.0,object


In [ ]:
out_csv = f"{CLEAN_DIR}/event_week7_clean.csv"
out_fix = f"{DOCS_DIR}/fix_log_week7.csv"

df3.to_csv(out_csv, index=False)
fix_log.to_csv(out_fix, index=False)

out_csv, out_fix


('/content/drive/MyDrive/ds-mini-project-2025/data_clean/event_week7_clean.csv',
 '/content/drive/MyDrive/ds-mini-project-2025/docs/fix_log_week7.csv')

In [ ]:
import matplotlib.pyplot as plt

miss_after = missingness_table(df3).reset_index().rename(columns={"index":"column"})
miss_after = miss_after.sort_values("missing_pct", ascending=False).head(10)

plt.figure()
plt.bar(miss_after["column"], miss_after["missing_pct"])
plt.xticks(rotation=45, ha="right")
plt.title("Top 10 Missingness % (After Week 7 Cleaning)")
plt.xlabel("Column")
plt.ylabel("Missing %")

fig_path = f"{FIG_DIR}/week7_missingness_top10.png"
plt.savefig(fig_path, bbox_inches="tight")
plt.close()

fig_path


'/content/drive/MyDrive/ds-mini-project-2025/figures/week7_missingness_top10.png'